In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

In [54]:
query = """
with test_data as (
    select
        t.uid,
        cast((julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch'))
            * 24 as integer) as delta,
        case 
            when t.first_commit_ts < t.first_view_ts then 'before'
            else 'after'
            end as time
    from test t
    join deadlines d on t.labname = d.labs
    where t.labname != 'project1'
),
both_time as (
    select td.uid
    from test_data td
    group by uid
    having count(distinct time) = 2
)

select
    time,
    avg(delta) as average
from test_data
where uid in both_time
group by time
"""

test_results = pd.read_sql_query(query, conn)
test_results

,time,average
0,after,-104.6000
1,before,-60.5625


In [56]:
query = """
with control_data as (
    select
        c.uid,
        cast((julianday(c.first_commit_ts) - julianday(d.deadlines, 'unixepoch'))
            * 24 as integer) as delta,
        case 
            when c.first_commit_ts < c.first_view_ts then 'before'
            else 'after'
            end as time
    from control c
    join deadlines d on c.labname = d.labs
    where c.labname != 'project1'
),
both_time as (
    select cd.uid
    from control_data cd
    group by uid
    having count(distinct time) = 2
)

select
    time,
    avg(delta) as average
from control_data
where uid in both_time
group by time
"""

test_results = pd.read_sql_query(query, conn)
test_results

,time,average
0,after,-117.636364
1,before,-99.464286


In [57]:
conn.close()